In [4]:
from selenium import webdriver as wd
import pandas as pd
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
# 각각의 플랫폼에 따른 폰트 설정을 위해서
import platform     # 플랫폼 관련 라이브러리
import time
from wordcloud import WordCloud
import seaborn as sns
import sys
import re

if platform.system() == 'Windows':
    font_path = "C:\Windows\Fonts\malgun.ttf"
    font_name = font_manager.FontProperties(fname=font_path).get_name()
    rc('font',family = font_name)
elif platform.system() == 'Darwin':
    
    rc('font',family='AppleGothic')
else :
    print('check your OS System')

In [ ]:
### 지도 시각화 : folium 라이브러리, kakao API 활용

In [77]:
### 1. 데이터 준비(1_crawling_raw.xlsx)
raw_total = pd.read_excel("data/1_crawling_raw.xlsx")
raw_total.head()

### 2. 위치정보 가져오기
raw_total['place']
location_counts = raw_total['place'].value_counts()
print(location_counts)


Jeju                    271
Jeju-do                 179
Jungle Book by Alice    108
Seogwipo                 66
제주도 크리스마스 박물관            59
                       ... 
제주마당                      1
형제섬                       1
동복해녀식당                    1
종로 광장시장                   1
디스이즈핫                     1
Name: place, Length: 1028, dtype: int64


In [85]:
### 위치정보를 DataFrame형태로 변환
location_counts_df = pd.DataFrame(location_counts)
location_counts_df.head()
location_counts_df.info()

### 위치정보를 엑셀파일로 저장
location_counts_df.to_excel('data/3_3_location_counts.xlsx')

<class 'pandas.core.frame.DataFrame'>
Index: 1028 entries, Jeju to 디스이즈핫
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   place   1028 non-null   int64
dtypes: int64(1)
memory usage: 16.1+ KB


In [79]:
### 위치(지명)만 별도로 종류를 확인
locations = list(location_counts.index)
locations

['Jeju',
 'Jeju-do',
 'Jungle Book by Alice',
 'Seogwipo',
 '제주도 크리스마스 박물관',
 '할로비치',
 '제주에인감귤밭',
 'Jeju Island',
 '성산일출봉 城山日出峰  Seongsan Ilchulbong',
 '폼포코식당_pompokokitchen',
 '1100고지',
 'Nimome',
 '월정리해변',
 '고집돌우럭중문점',
 '석부작박물관',
 '제주도 애월읍',
 '알뜨르 비행장',
 '카멜리아 힐',
 '제주레포츠랜드',
 '밥깡패',
 '제주 함덕 서우봉 해변',
 '제주도 서귀포 중문관광단지 濟州島西歸浦中文觀光團地',
 '제주고궁한복카페 jeju gogung hanbok studio',
 '위미동백나무군락지',
 '제주신화월드 Jeju Shinhwa World',
 'Woljeongri Beach',
 '협재해변 Beach',
 '연동 바오젠 거리',
 '김녕미로공원 Jeju Kimnyoung Maze Park',
 '고래배꼽',
 '제주 성산 신산포구자연산횟집 회포장',
 '새별오름',
 '용머리해안',
 '용눈이오름',
 '캔디원',
 '사려니숲길',
 '하이엔드 제주',
 '제주빅볼랜드',
 '서귀포 느영나영 게스트하우스',
 '휴애리 자연생활공원',
 '옹포리',
 '삼무공원',
 '마마뜰',
 '제주어린왕자게스트하우스',
 '위미동백나무군락',
 '제주커피박물관 Baum',
 'Jeju Aewol',
 '한라산',
 '위미2리 동백군락지',
 'Jeju Island 제주특별자치도 济州道',
 '광치기해변',
 '서우봉',
 '제주해남',
 '한라산 (漢拏山, Hallasan)',
 '카페한라산',
 '성이시돌목장',
 '제주 송악산',
 '옹포별장가든',
 '월정리카페콧수염',
 '牛岛 Udo Island 우도',
 '곽지해수욕장',
 '카페브리프',
 '아줄레주',
 '에코랜드',
 '산굼부리',
 '제주 꿈꾸는고래 스쿠버&게스트하우스',
 'Terarosa - 테라로사',

In [7]:
### kakao API 사용(로컬/지도 API) : 장소 검색
import requests

searching = "스타벅스 합정점"
url = "https://dapi.kakao.com/v2/local/search/keyword.json?query={}".format(searching)
header = {
    "Authorization": "KakaoAK eddcde87a3e7085c703e3f35dde1cd2f"
    # 입력시 반드시 KakaoAK뒤에 한칸 띄우고 API키를 적어야한다
}
places = requests.get(url, headers=header).json()['documents']
places

[{'address_name': '서울 마포구 합정동 472',
  'category_group_code': 'CE7',
  'category_group_name': '카페',
  'category_name': '음식점 > 카페 > 커피전문점 > 스타벅스',
  'distance': '',
  'id': '2057327896',
  'phone': '1522-3232',
  'place_name': '스타벅스 합정점',
  'place_url': 'http://place.map.kakao.com/2057327896',
  'road_address_name': '서울 마포구 월드컵로1길 14',
  'x': '126.91253700818196',
  'y': '37.54994959743763'}]

In [8]:
### 카카오 API를 활용한 장소 검색 함수 만들기
## 인자 : searching
## place[첫번째], name-장소이름, x, y
## 반환값: [name, x, y, searching]
def find_place(searching):
    ## kakao api 접속 url
    url = "https://dapi.kakao.com/v2/local/search/keyword.json?query={}".format(searching)
    
    ## headers 입력
    header = {
    "Authorization": "KakaoAK eddcde87a3e7085c703e3f35dde1cd2f"
    # 입력시 반드시 KakaoAK뒤에 한칸 띄우고 API키를 적어야한다
    }
    
    ## API 요청 및 정보 처리
    places = requests.get(url, headers=header).json()['documents']
    
    ## 필요 정보를 선택하여 반환 처리
    place = places[0]
    name = place['place_name']
    x = place['x']
    y = place['y']
    data = [name, x, y, searching]
    return data

    

In [9]:
## 함수 확인
data = find_place("제주공항")
data

['제주국제공항', '126.492769004244', '33.5070789578184', '제주공항']

In [33]:
### 반복작업 진행시 진행바 표시하기 위한 라이브러리 : tqdm 활용
#!pip list | findstr tqdm
#pip install tqdm

You should consider upgrading via the 'C:\Program Files\Python310\python.exe -m pip install --upgrade pip' command.


In [43]:
pip install ipywidgets

Defaulting to user installation because normal site-packages is not writeable
     -------------------------------------- 123.4/123.4 KB 7.5 MB/s eta 0:00:00
     -------------------------------------- 245.1/245.1 KB 3.8 MB/s eta 0:00:00
     ---------------------------------------- 1.6/1.6 MB 7.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Program Files\Python310\python.exe -m pip install --upgrade pip' command.


In [10]:
from tqdm.notebook import tqdm

In [80]:
locations_inform = []
for location in tqdm(locations):
    try:
        data = find_place(location)
        locations_inform.append(data)
        time.sleep(0.5)
    except:
        pass
locations_inform

  0%|          | 0/1028 [00:00<?, ?it/s]

[['Jeju in aA', '126.83650369902168', '33.53833652360498', 'Jeju'],
 ['서귀포잠수함', '126.558616052674', '33.2393033784206', 'Seogwipo'],
 ['바이나흐튼 크리스마스박물관', '126.32785786016811', '33.2915720831759', '제주도 크리스마스 박물관'],
 ['할로비치 본점', '126.2815248933638', '33.30832616522123', '할로비치'],
 ['제주에인감귤밭', '126.5390139270361', '33.25656069063887', '제주에인감귤밭'],
 ['제주도', '126.54587355630036', '33.379777816446165', 'Jeju Island'],
 ['한라산1100고지', '126.462219691112', '33.3580781709788', '1100고지'],
 ['월정리해수욕장', '126.795805057888', '33.556469394054', '월정리해변'],
 ['고집돌우럭 중문점', '126.416704762779', '33.2579811121134', '고집돌우럭중문점'],
 ['석부작박물관', '126.53652102219156', '33.25045551460648', '석부작박물관'],
 ['올레길 16코스(고내-광령 올레)', '126.38979561557086', '33.46649759191228', '제주도 애월읍'],
 ['알뜨르비행장', '126.271527320164', '33.2047161778831', '알뜨르 비행장'],
 ['카멜리아힐', '126.370009467525', '33.2896344311608', '카멜리아 힐'],
 ['제주레포츠랜드', '126.6387551242765', '33.48206819907232', '제주레포츠랜드'],
 ['밥깡패', '126.26342309622974', '33.41011846869669', '

In [84]:
### 생성된 정보를 df로 만들어서 엑셀로 저장
locations_inform_df = pd.DataFrame(locations_inform)
locations_inform_df.columns = ['name_official','경도','위도','인스타위치명']
locations_inform_df.to_excel('data/3_locations.xlsx',index=False)

In [86]:
### 인스타 게시량 파일(3_3_location_counts.xlsx), 위치정보 데이터 불러오기
location_counts_df = pd.read_excel('data/3_3_location_counts.xlsx',index_col=0)
location_inform_df = pd.read_excel('data/3_locations.xlsx')

### 병합하기(위치 데이터 병합)
location_data = pd.merge(left=location_inform_df, right=location_counts_df, how='inner', left_on='name_official', right_index=True)
# inner : 교집합만 출력
location_data.info()
location_data.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 407 entries, 1 to 798
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   name_official  407 non-null    object 
 1   경도             407 non-null    float64
 2   위도             407 non-null    float64
 3   인스타위치명         407 non-null    object 
 4   place          407 non-null    int64  
dtypes: float64(2), int64(1), object(2)
memory usage: 19.1+ KB


,name_official,경도,위도,인스타위치명,place
1,서귀포잠수함,126.558616,33.239303,Seogwipo,1
620,서귀포잠수함,126.558616,33.239303,서귀포잠수함,1
4,제주에인감귤밭,126.539014,33.256561,제주에인감귤밭,48
5,제주도,126.545874,33.379778,Jeju Island,4
84,제주도,126.545874,33.379778,제주도,4


In [87]:
### 데이터 중복 점검하기
location_data['name_official'].value_counts()

제주동문재래시장    4
오설록티뮤지엄     3
자매국수 본점     3
서귀포잠수함      2
빌라드아토       2
           ..
쉐프부부        1
이게밥도둑       1
숨비아일랜드      1
삼공주회센터      1
디스이즈핫       1
Name: name_official, Length: 376, dtype: int64

In [88]:
### 장소 이름을 기준으로 병합(pivot_table)
location_data = location_data.pivot_table(index=['name_official','경도','위도'], values='place',aggfunc="sum")
location_data

,,,place
name_official,경도,위도,
73st,126.455109,33.456989,1
가드망제,126.532954,33.500991,1
가배,126.951868,37.738173,1
가시식당,126.771438,33.353060,2
가시아방국수,126.918065,33.438605,2
...,...,...,...
황궁쟁반짜장,127.394311,36.338770,1
황우지해안,126.550528,33.241467,2
훈남횟집,126.667858,33.542086,1


In [89]:
### 병합 데이터를 저장하기
location_data.to_excel('data/3_location_inform.xlsx')

In [90]:
### 위에서 사용한 location_data(data/3_location_inform.xlsx)
location_data = pd.read_excel('data/3_location_inform.xlsx')
location_data.info()
location_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 377 entries, 0 to 376
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   name_official  376 non-null    object 
 1   경도             377 non-null    float64
 2   위도             377 non-null    float64
 3   place          377 non-null    int64  
dtypes: float64(2), int64(1), object(1)
memory usage: 11.9+ KB


,name_official,경도,위도,place
0,73st,126.455109,33.456989,1
1,가드망제,126.532954,33.500991,1
2,가배,126.951868,37.738173,1
3,가시식당,126.771438,33.353060,2
4,가시아방국수,126.918065,33.438605,2


In [26]:
### folium을 이용한 지도 시각화
#pip install folium

SyntaxError: invalid syntax (3455994747.py, line 2)

In [28]:
!pip install folium

Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the 'C:\Program Files\Python310\python.exe -m pip install --upgrade pip' command.


In [91]:
import folium

MT_Hanla = [33.362500,126.533694]
map_jeju = folium.Map(location=MT_Hanla,zoom_start=11)

for i in range(len(location_data)):
    name = location_data['name_official'][i]    # 공식명칭
    count = location_data['place'][i]           # 게시글 개수
    size = int(count)*2
    long = float(location_data['위도'][i])
    let = float(location_data['경도'][i])
    folium.CircleMarker((long,let), radius=size, color="red", popup=name).add_to(map_jeju)
map_jeju

In [92]:
map_jeju.save('data/3_jeju.html')

In [93]:
## 마커 집합을 통한 지도표시 
''' (location_data)
name_offial	경도	       위도	      place
73st	    126.455109	  33.456989	1
가드망제	  126.532954	33.500991  1
가배	     126.951868	   37.738171   1
가시식당	  126.771438	33.353060	2
가시아방국수   126.918065	 33.438605	2 '''
from folium.plugins import MarkerCluster

locations=[] 
names=[] 

for i in range(len(location_data)): 
    data= location_data.iloc[i] # 행하나씩
    locations.append((float(data['위도']),float(data['경도'])))  #위도,경도 순으로 위치를 저장  
    names.append(data['name_official'])

MT_Hanla = [33.362500,126.533694] 
map_jeju2 = folium.Map(location=MT_Hanla,zoom_start=11) 

marker_cluster = MarkerCluster( 
    locations= locations, popups=names,
    name= 'Jeju',
    overlay= True, 
    control= True     
                              )

marker_cluster.add_to(map_jeju2) 
folium.LayerControl().add_to(map_jeju2)

map_jeju2

In [52]:
map_jeju2.save('data/3_jeju_cluster.html')

In [66]:
## 특정 단어 포함 게시글 찾기

# import pandas as pd
raw_total = pd.read_excel('data/1_crawling_raw.xlsx')
raw_total.head()

,content,date,like,place,tags
0,더 늦기전에 제주도로 떠나쟈😍.#제주핫플레이스 간단히 1탄 모아봄!#제주여행 요 필...,2018-12-07,"1,402",NaN,"['#제주핫플레이스', '#제주여행', '#제주여행', '#제주도여행', '#제주가..."
1,12월에 제주도를 가야하는이유🍊.#제주핫플 모음 끝판왕😍이거 하나면 겨울 #제주여행...,2018-12-03,368,NaN,"['#제주핫플', '#제주여행', '#제주', '#제주도', '#제주도맛집', '#..."
2,11월 놓치지 말아야 할 제주 관광.가을바람이 최고조에 이르는 11월추운 겨울을 앞...,2018-11-02,166,NaN,"['#honestin', '#어니스틴', '#제주여행', '#제주', '#제주도',..."
3,국민학교세대#제주관광#제주살이#제주이주민#아라동주민#삼남매집 #새해첫날#드라이브#명...,2019-01-01,28,명월국민학교,"['#제주관광', '#제주살이', '#제주이주민', '#아라동주민', '#삼남매집'..."
4,#제주관광 #제주 #돔나이트 #스트레스 #풀자 #춤추며 #땀날려 #가끔은괜찮아 #인...,2019-01-01,12,NaN,"['#제주관광', '#제주', '#돔나이트', '#스트레스', '#풀자', '#춤추..."


In [75]:
## 단어선택하기
# 예시 "해돋이"
selected_word = "섭지코지"

check_list = []
for content in raw_total['content']:
    if selected_word in content:
        check_list.append(True)
    else:
        check_list.append(False)

select_df = raw_total[check_list]
select_df.head()

### 선택한 데이터 확인
for i in select_df.index:
    print(select_df.loc[i,'content'])
    print("-"*50)
select_df.head()

안녕하세요. 폼포코식당입니다!_  정신없는 연말.. 피드가 뜸했쥬?!바쁘디 바쁜 하루하루를 보내고,요렇게 이쁘게 정리까지 해주시는 손님들에게감사함을 느끼며 오늘 하루도 마무리했습니다.내일 31일은 폼포코 정기휴무이며,1월 1일은 외부행사로 인해 하루 더 쉬어갑니다.다들 2019년 새해 복 많이 받으세요.🙏🏻2018년 한해도 많은 사랑 주셔서 감사해요-❣️..#제주도 #제주 #jeju #서귀포 #seogwipo#성산일출봉 #섭지코지 #서귀포떡볶이 #성산떡볶이#성산맛집 #고성리맛집 #서귀포분식 #성산심야식당#심야식당 #플레이스캠프제주 #playce #폼포코식당#폼포코 #pompokokitchen #모찌동 #쿠시카츠#서귀포이자카야 #고성리술집 #제주동쪽맛집 #제주동쪽 #제주여행 #제주관광 #제주도민 #혼술🍺
--------------------------------------------------
안녕하세요. 폼포코식당입니다!_  오늘은 크리스마스이브🎅🏻❣️폼포코 정기휴무날 깜짝으로 오픈했음에도 불구하고,많은 분들이 찾아주셔서 너무 감사합니다.손님들이 "메리크리스마스-" 인사해주시는게바쁘지만 너무너무 행봌했어요.내일도 폼포코 문 활짝 ! 🙌🏻 또 만나요~..#제주도 #제주 #jeju #서귀포 #seogwipo#성산일출봉 #섭지코지 #서귀포떡볶이 #성산떡볶이#성산맛집 #고성리맛집 #서귀포분식 #성산심야식당#심야식당 #플레이스캠프제주 #playce #폼포코식당#폼포코 #pompokokitchen #모찌동 #쿠시카츠#서귀포이자카야 #고성리술집 #제주동쪽맛집 #제주동쪽 #제주여행 #제주관광 #제주도민 #혼술🍺
--------------------------------------------------
안녕하세요. 폼포코식당입니다!_  갑자기 느끼한게 땡기는 날🎅🏻🤟🏽속은 촉촉, 치즈 줄줄~ 오꼬노미야끼와샴페인 느낌 물씬 나는 미오 스파클링 한잔사랑하는 사람과 함께 먹기 딱!!! 🍝🥂..#제주도 #제주 #jeju #서귀포 #seogwipo#성산일출봉 #섭지코지 #서귀포떡볶이 

,content,date,like,place,tags
89,안녕하세요. 폼포코식당입니다!_ 정신없는 연말.. 피드가 뜸했쥬?!바쁘디 바쁜 하...,2018-12-30,42,폼포코식당_pompokokitchen,"['#제주도', '#제주', '#jeju', '#서귀포', '#seogwipo', ..."
252,안녕하세요. 폼포코식당입니다!_ 오늘은 크리스마스이브🎅🏻❣️폼포코 정기휴무날 깜짝...,2018-12-24,45,폼포코식당_pompokokitchen,"['#제주도', '#제주', '#jeju', '#서귀포', '#seogwipo', ..."
361,"안녕하세요. 폼포코식당입니다!_ 갑자기 느끼한게 땡기는 날🎅🏻🤟🏽속은 촉촉, 치즈...",2018-12-21,42,폼포코식당_pompokokitchen,"['#제주도', '#제주', '#jeju', '#서귀포', '#seogwipo', ..."
379,제주 너~ 비와도 괜찮다.#비오는날제주 #섭지코지 #한라산 #1100고지 #이중섭거...,2018-12-20,42,Jeju-do,"['#비오는날제주', '#섭지코지', '#한라산', '#1100고지', '#이중섭거..."
383,안녕하세요. 폼포코식당입니다!_ 와가야노아마구치+냄비밀푀유나베 SET 2개 남았습니...,2018-12-20,40,폼포코식당_pompokokitchen,"['#제주도', '#제주', '#jeju', '#서귀포', '#seogwipo', ..."


In [76]:
fpath = f"data/4_select_data_{selected_word}.xlsx"
select_df.to_excel(fpath)

In [ ]:
## 여러개의 단어 선택/추출/저장 하세요
# 해돋이, 박물관,힐링,게스트하우스,섭지코지
# 각 선택단어를 추출하여 저장(4_select_data{선택단어}.xlsx로 저장)